In [ ]:
# !git clone https://github.com/vincentnam/ProjetInterpromo2020G8.git

In [5]:
from layout_processing.blackwhite import BlackWhite
from layout_processing.colourpipelinezone import ColourPipelineZones
from layout_processing.pipeline import Pipeline
from layout_processing.removedoubleseat import RemoveDoubleSeat
from layout_processing.seatfinder import SeatFinder
from layout_processing.segmentationzone import SegmentationZone

In [ ]:
base_data_path = "../../M2_DATA_PIPELINE/"

# Pipeline Seat

In [ ]:
pipeline_seat = Pipeline(base_data_path,["Aer_Lingus_Airbus_A330-300_A_plane6.jpg"])
pipeline_seat.add_processes([BlackWhite,SeatFinder, RemoveDoubleSeat])

In [ ]:
pipeline_seat.run_pipeline(1)

# Pipeline Zone

In [ ]:
pipeline_zone = Pipeline(base_data_path,["Aer_Lingus_Airbus_A330-300_A_plane6.jpg"])
pipeline_zone.add_processes([ColourPipelineZones, BlackWhite, SegmentationZone])
pipeline_zone.run_pipeline(1)

In [ ]:
import cv2 as cv 
import matplotlib.pyplot as plt 


def to_rgb(image):
        return cv.cvtColor(image, cv.COLOR_BGR2RGB)
    
def draw_contour(image, name_img, pipeline_zone, pipeline_seat):
    
    image = to_rgb(image) # change color postion BGR to RGB
    
    # 
    dict_colour_seat = {'BUSINESS': (255, 0, 0),
                     'ECONOMY': (255, 0, 255),
                     'FIRST': (0, 255, 0),
                     'PREMIUM': (255, 255, 0)}
    
    # for each rectangles it will draw it on the image
    for zone in pipeline_zone[name_img]:
        for rect in pipeline_zone[name_img][zone]:
            y, x = rect[0], rect[1]
            w, h = abs(x-rect[3]), abs(y-rect[2])
            image = cv.rectangle(image, (x,y), (x+w, y+h),(0,0,255), 2)
            
    for category in pipeline_seat[name_img]:
        for seat in pipeline_seat[name_img][category]:
            x , y = seat[0], seat[1]
            h, w = seat[2], seat[3]
            image = cv.rectangle(image, (x,y), (x+w, y+h),dict_colour_seat[category], 2)
     
    plt.figure(figsize=(20,40))
    plt.imshow(image)

image = cv.imread(base_data_path + '/All Data'+
                 '/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-300_A_plane6.jpg')

draw_contour(image,"Aer_Lingus_Airbus_A330-300_A_plane6.jpg", pipeline_zone.json, pipeline_seat.json)

## Pipeline Merge

In [ ]:
from layout_processing.distpipeline import DistPipeline

In [ ]:
dist_pipeline = DistPipeline(pipeline_seat, pipeline_zone)

In [ ]:
json_dist = dist_pipeline.to_json_simple_distance(dist_pipeline.pipeline.json, dist_pipeline.pipeline_zone.json)
print(json_dist)